In [1]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
load_dotenv()

es = Elasticsearch("https://"+os.getenv("ELASTICSEARCH_HOST")+":"+os.getenv("ELASTICSEARCH_PORT"),http_auth=(os.getenv("ELASTICSEARCH_USER"),os.getenv("ELASTICSEARCH_PASSWORD")),verify_certs=False)
es.ping()

/Users/dsraju/Work/Python/elasticsearch/synonyms/.venv/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/var/folders/r7/18k8z10d2dgbvn5zl0vhfynr0000gn/T/ipykernel_6525/2445713845.py:8: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch("https://"+os.getenv("ELASTICSEARCH_HOST")+":"+os.getenv("ELASTICSEARCH_PORT"),http_auth=(os.getenv("ELASTICSEARCH_USER"),os.getenv("ELASTICSEARCH_PASSWORD")),verify_certs=False)


True

In [2]:
documents = [
    {"_index": "file_synonym_index", "_source": {"topic": "Electric vehicles"}},
    {"_index": "file_synonym_index", "_source": {"topic": "fuel efficiency"}},
    {"_index": "file_synonym_index", "_source": {"topic": "cars"}},
    {"_index": "file_synonym_index", "_source": {"topic": "maintenance"}},
    {"_index": "file_synonym_index", "_source": {"topic": "technology"}},
    {"_index": "file_synonym_index", "_source": {"topic": "fuel prices"}},
    {"_index": "file_synonym_index", "_source": {"topic": "economical"}},
    {"_index": "file_synonym_index", "_source": {"topic": "Car manufacturers"}},
    {"_index": "file_synonym_index", "_source": {"topic": "research and development"}},
    {"_index": "file_synonym_index", "_source": {"topic": "petrol stations"}}
]

In [3]:
# Create the synonym sets
resp = es.synonyms.put_synonym(
    id="automobile-synonym-set",
    synonyms_set=[
            {
            "id": "auto-1",
            "synonyms": "Electric vehicles,Electric vehicle, EV, EVs"
            },
            {
            "id": "auto-2",
            "synonyms": "fuel => petrol, gas"
            },
            {
            "id": "auto-3",
            "synonyms": "servicing, repair => maintenance"
            },
            {
            "id": "auto-4",  
            "synonyms": "economical => cost-effective"
            },
            {
            "id": "auto-5",  
            "synonyms": "technology, tech"
            },
            {
            "id": "auto-6",  
            "synonyms": "research and development, R&D"
            },
            {
            "id": "auto-7",  
            "synonyms": "bunk => station"
            },
            {
            "id": "auto-8",  
            "synonyms": "howdy => stations"
            }
        ],
)

In [4]:
settings = {
    "analysis": {
        "filter": {
            "synonyms_filter": {
                "type": "synonym_graph",
                "synonyms_set": "automobile-synonym-set",
                "updateable": True
            }
        },
        "analyzer": {
            "index_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": ["lowercase"]
            },
            "search_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": ["lowercase", "synonyms_filter"]
            }
        }
    }
}
mappings = {
    "properties": {
        "topic": {
            "type": "text",
            "analyzer": "index_analyzer",
            "search_analyzer": "search_analyzer"
        }
    }
}
body = {
    "settings": settings,
    "mappings": mappings
}

if es.indices.exists(index="file_synonym_index"):
    es.indices.delete(index="file_synonym_index")
es.indices.create(index="file_synonym_index",body=body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'file_synonym_index'})

In [5]:
from elasticsearch import helpers

helpers.bulk(es, documents)

(10, [])

In [6]:
def search_index(index_name, query):
    print("Searching for: ", query)
    response = es.search(index=index_name, 
                    body={"query": 
                          {"match": 
                           {"topic": query}}})
    for res in response.get('hits')['hits']:
        print(res.get('_source').get('topic'))

In [7]:
search_index("file_synonym_index", "EVs")

Searching for:  EVs
Electric vehicles


In [8]:
search_index("file_synonym_index", "EV")

Searching for:  EV
Electric vehicles


In [12]:
search_index("file_synonym_index", "fuel")

Searching for:  fuel
petrol stations


In [46]:
search_index("file_synonym_index", "howdy")

Searching for:  howdy
petrol stations


In [30]:
search_index("file_synonym_index", "servicing")

Searching for:  servicing
maintenance


In [31]:
search_index("file_synonym_index", "repair")

Searching for:  repair
maintenance


In [32]:
search_index("file_synonym_index", "R&D")

Searching for:  R&D
research and development


In [33]:
search_index("file_synonym_index", "tech")

Searching for:  tech
technology
